In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
#from tensorflow.keras.callbacks import TensorBoard
import pickle
import numpy as np
import time
import pandas as pd

df_X_test = pickle.load(open("df_x_test.pickle","rb"))
df_fake_trips = pickle.load(open("df_Y_test_fake_Trips.pickle","rb"))

print(df_X_test.head())

                                  hash  \
0  00032f51796fd5437b238e3a9823d13d_31   
1  000479418b5561ab694a2870cc04fd43_25   
2   0005401ceddaf27a9b7f0d42ef1fbe95_1   
3   00063a4f6c12e1e4de7d876580620667_3   
4  000d479078af08618bddc7f09082b8c3_11   

                                       traj_1  time_entry_x  time_exit_x  \
0  traj_00032f51796fd5437b238e3a9823d13d_31_0       42197.0      42617.0   
1  traj_000479418b5561ab694a2870cc04fd43_25_0       29303.0      30008.0   
2   traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_0       33003.0      33003.0   
3   traj_00063a4f6c12e1e4de7d876580620667_3_1       31718.0      34436.0   
4  traj_000d479078af08618bddc7f09082b8c3_11_0       49352.0      50110.0   

      x_entry_x     y_entry_x      x_exit_x      y_exit_x  Good_hash_x  \
0  3.773413e+06 -1.909828e+07  3.773111e+06 -1.914508e+07         True   
1  3.771380e+06 -1.933274e+07  3.769933e+06 -1.934095e+07         True   
2  3.751328e+06 -1.916236e+07  3.751328e+06 -1.916236e+07         Tr

In [2]:

print(df_fake_trips.head())
df_fake_trips.describe()

                                  trajectory_id  incity
10   traj_000506a39775e5bca661ac80e3f466eb_29_5    True
23   traj_0006535be25bb52dd06983447880c964_5_12   False
27   traj_0006f84bb33ec929d1cda7686f861d0a_31_3    True
31  traj_00093ae562586aed0e053b8431e8ace4_23_10   False
34   traj_000c739e444a70e1804d757a0580caaa_31_3    True


,trajectory_id,incity
count,16841,16841
unique,16841,2
top,traj_919287e327e0adad48d2808e4580ee03_29_7,False
freq,1,11980


In [3]:
#convert to numpy arrays

columns_np = ['Xi1','Xf1','Yi1','Yf1','Ti1','Tf1',
             'Xi2','Xf2','Yi2','Yf2','Ti2','Tf2',
             'Xi3','Yi3','Ti3','Tf3']
x_test = df_X_test[columns_np].values


print(x_test[0:5])

[[0.89790318 0.88951588 0.83652352 0.69898662 0.73258681 0.73987847
  0.89710145 0.89007092 0.79196034 0.70024029 0.83911458 0.85751736
  0.88973173 0.69951449 0.94118056 0.94847222]
 [0.84156645 0.80148011 0.14747403 0.12333793 0.50873264 0.52097222
  0.84156645 0.80286772 0.14747403 0.11833687 0.75873264 0.76395833
  0.80274437 0.1221133  0.96786458 0.96972222]
 [0.28612396 0.28612396 0.64820893 0.64820893 0.57296875 0.57296875
  0.28670984 0.28670984 0.64678637 0.64678637 0.60171875 0.60171875
  0.54033302 0.08203088 0.97427083 0.98133681]
 [0.17631822 0.92269504 0.3069781  0.801938   0.55065972 0.59784722
  0.97687327 0.70074006 0.58787107 0.62490518 0.89409722 0.91149306
  0.7013876  0.62536888 0.93137153 0.94295139]
 [0.10687635 0.10641381 0.25749546 0.25280017 0.85680556 0.86996528
  0.10564292 0.73919211 0.17917786 0.52520836 0.91248264 0.93447917
  0.70653716 0.53239939 0.93552083 0.94131944]]


In [4]:
# load model
new_model = tf.keras.models.load_model('first_Dense_V5_1.model') 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [7]:
#make predictions
predictions_float = new_model.predict(x_test) # predict the output using the model
print(predictions_float[0:10])

[[0.08410946]
 [0.08531079]
 [0.21535638]
 [0.2394079 ]
 [0.3310572 ]
 [0.9013163 ]
 [0.31809956]
 [0.08104104]
 [0.17497915]
 [0.7786213 ]]


In [8]:
threshold = 0.657
predictions=predictions_float>=threshold
predictions=predictions*1
predictions

C:\Users\sofia\AppData\Local\conda\conda\envs\mlcc\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater_equal
  


array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [9]:
#put predictions back into the DF (order shouldnt have changed)
df_pred=pd.DataFrame({'trajectory_id':df_X_test['traj_3'],'incity':predictions[:,0]})

print(df_pred.head(10))
df_pred.describe()

                                 trajectory_id  incity
0   traj_00032f51796fd5437b238e3a9823d13d_31_5       0
1  traj_000479418b5561ab694a2870cc04fd43_25_10       0
2    traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4       0
3    traj_00063a4f6c12e1e4de7d876580620667_3_4       0
4   traj_000d479078af08618bddc7f09082b8c3_11_6       0
5   traj_000efd6f95db39d6ba35027f02b5628f_9_20       1
6  traj_000feace3a33be9245e2783cdff467a8_15_12       0
7    traj_0011b48d0cf4f962833edd701afaf419_3_7       0
8   traj_001484cf0f956c316070be4b4c6352ff_29_7       0
9    traj_001d29060cfe35996cfeb141210ecbeb_9_2       1


,incity
count,16674.000000
mean,0.290332
std,0.453929
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [10]:
#combine with predictions about the fake trips, what about ordering compared to the OG test?
df_Y_test=pd.concat([df_fake_trips,df_pred])
print(df_Y_test.head(10))
df_Y_test.describe()

                                   trajectory_id  incity
10    traj_000506a39775e5bca661ac80e3f466eb_29_5       1
23    traj_0006535be25bb52dd06983447880c964_5_12       0
27    traj_0006f84bb33ec929d1cda7686f861d0a_31_3       1
31   traj_00093ae562586aed0e053b8431e8ace4_23_10       0
34    traj_000c739e444a70e1804d757a0580caaa_31_3       1
45   traj_000d6fba12a7d06defe90e3c9162c11e_29_11       1
87    traj_002447d99e8f6c3bc6c1329140979a91_25_7       0
94    traj_002531dea45cd4059e92099630ebf775_3_10       0
103   traj_0026d48a63f219dc359f1131805909bf_31_5       0
111   traj_002f7d1d09d2af27ee5a6d0fa4e91c96_29_8       1


,incity
count,33515.000000
mean,0.289482
std,0.453529
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [11]:
df_Y_test.rename(columns={'incity':'target','trajectory_id':'id'} , inplace=True)
print(df_Y_test.head(10))

                                              id  target
10    traj_000506a39775e5bca661ac80e3f466eb_29_5       1
23    traj_0006535be25bb52dd06983447880c964_5_12       0
27    traj_0006f84bb33ec929d1cda7686f861d0a_31_3       1
31   traj_00093ae562586aed0e053b8431e8ace4_23_10       0
34    traj_000c739e444a70e1804d757a0580caaa_31_3       1
45   traj_000d6fba12a7d06defe90e3c9162c11e_29_11       1
87    traj_002447d99e8f6c3bc6c1329140979a91_25_7       0
94    traj_002531dea45cd4059e92099630ebf775_3_10       0
103   traj_0026d48a63f219dc359f1131805909bf_31_5       0
111   traj_002f7d1d09d2af27ee5a6d0fa4e91c96_29_8       1


test_output = df_Y_test[['hash','incity']].values

print(test_output[0:5])

In [12]:
df_Y_test.to_csv('Y_Prediction_V5_1.csv', sep=',',index=False)